In [1]:
from math import sqrt, ceil
import numpy as np
from scipy.sparse.linalg import norm
import matplotlib.pyplot as plt
import pandas as pd
import cirq
import openfermion as of
from openfermionpyscf import run_pyscf
import qiskit
from qiskit.circuit.library import PauliEvolutionGate, phase_estimation
from qiskit.synthesis import LieTrotter
from qiskit import transpile
from qpe_trotter import get_gate_counts
from convert import cirq_pauli_sum_to_qiskit_pauli_op

In [2]:
def eps2_bound(hamiltonian: cirq.PauliSum) -> float:
    """Get a bound on eps2."""

    M = len(hamiltonian)
    coeffs = np.array([ps.coefficient for ps in hamiltonian])
    i_max = np.argmax(np.abs(coeffs))
    max_coeff = coeffs[i_max]
    n_summands = M
    return (-1. / 24) * 0.5 * max_coeff ** 3 * n_summands ** 3

In [4]:
data = np.load("po3_1w_product_22o.npz")

# Get core energy and one-/two-body tensors
ecore = data['ECORE']
h1 = np.array(data['H1'])
h2 = np.array(data['H2'])
norb = data["NORB"]

# Get number of alpha and beta electrons
n_a = int(data["NELEC"]/2)
n_b = int(data["NELEC"]/2)

h2 = 0.5 * np.asarray(h2.transpose(0, 2, 3, 1), order="C")
h1_new, h2_new = of.chem.molecular_data.spinorb_from_spatial(h1, h2)

# Get the Hamiltonian and transform it to FermionOperator
h = of.InteractionOperator(ecore.item(), h1_new, h2_new)

hamiltonian = of.get_fermion_operator(h)
# hamiltonian.compress(0.001)

nqubits = of.utils.count_qubits(hamiltonian)
nterms = len(hamiltonian.terms)

print(f"Hamiltonian acts on {nqubits} qubit(s) and has {nterms} term(s).")

KeyError: 'ECORE is not a file in the archive'